In [373]:
import pandas as pd
import numpy as np


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop


from sklearn.preprocessing import MinMaxScaler,OneHotEncoder
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error, r2_score

In [374]:
#Load dataset into a pandas dataframe
df = pd.read_csv("../data/Cleaned data/data.csv") 

df.dropna(inplace = True)

df.head()

,YEAR,MONTH,DAY,HUMIDITY,WINDSPEED,DATE,TOTALDEMAND,HOLIDAY,MIN,MAX,RAIN,SOLAR,RRP,FORECASTDEMAND,OUTPUT,MONTHDATE,WEEKDAY,WEEKEND,TEMPAVE
0,2016,1,1,0.656341,15.902439,2016-01-01,6853.633437,2.0,15.3,28.6,0.0,32.2,38.472917,6665.366167,46.93,01-2016,4,0,21.95
1,2016,1,2,0.656341,15.902439,2016-01-02,6727.613958,0.0,15.9,26.1,0.0,21.7,36.907292,6236.849955,46.93,01-2016,5,1,21.00
2,2016,1,3,0.688837,14.488372,2016-01-03,6616.406076,0.0,17.5,25.6,0.0,10.3,31.997083,6551.924748,46.93,01-2016,6,1,21.55
3,2016,1,4,0.679545,22.477273,2016-01-04,7367.750278,0.0,18.2,23.6,14.0,6.4,33.424583,6729.993123,46.93,01-2016,0,0,20.90
4,2016,1,5,0.768837,22.581395,2016-01-05,7462.242014,0.0,17.6,20.5,39.0,4.4,33.053958,7333.898202,46.93,01-2016,1,0,19.05


In [375]:
#X = df.iloc[:,[3,5,9,16]]
#y = df.iloc[:,10]

df = df.drop(['DATE','MIN','MAX','FORECASTDEMAND', 'MONTHDATE','WEEKEND'], axis=1)

#y = df['FORECASTDEMAND']

df.head()


,YEAR,MONTH,DAY,HUMIDITY,WINDSPEED,TOTALDEMAND,HOLIDAY,RAIN,SOLAR,RRP,OUTPUT,WEEKDAY,TEMPAVE
0,2016,1,1,0.656341,15.902439,6853.633437,2.0,0.0,32.2,38.472917,46.93,4,21.95
1,2016,1,2,0.656341,15.902439,6727.613958,0.0,0.0,21.7,36.907292,46.93,5,21.00
2,2016,1,3,0.688837,14.488372,6616.406076,0.0,0.0,10.3,31.997083,46.93,6,21.55
3,2016,1,4,0.679545,22.477273,7367.750278,0.0,14.0,6.4,33.424583,46.93,0,20.90
4,2016,1,5,0.768837,22.581395,7462.242014,0.0,39.0,4.4,33.053958,46.93,1,19.05


In [376]:
# Onehot Encoding for categorial data (Weekday)


# Select the "WEEKDAY" column and create a new dataframe
weekday_df = df[['WEEKDAY']]

# Create a one-hot encoder object

encoder = OneHotEncoder(categories='auto')

# Fit and transform the weekday data
weekday_encoded = encoder.fit_transform(df[['WEEKDAY']]).toarray()

# Create a new dataframe with the encoded weekday data
weekday_df = pd.DataFrame(weekday_encoded, columns=['MON', 'TUE', 'WED', 'THU', 'FRI', 'SAT', 'SUN'])

#Drop weekday column
df = df.drop(['WEEKDAY'], axis=1)

# Concatenate the original dataframe with the encoded weekday dataframe
df = pd.concat([df, weekday_df], axis=1)


df.head()

,YEAR,MONTH,DAY,HUMIDITY,WINDSPEED,TOTALDEMAND,HOLIDAY,RAIN,SOLAR,RRP,OUTPUT,TEMPAVE,MON,TUE,WED,THU,FRI,SAT,SUN
0,2016,1,1,0.656341,15.902439,6853.633437,2.0,0.0,32.2,38.472917,46.93,21.95,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,2016,1,2,0.656341,15.902439,6727.613958,0.0,0.0,21.7,36.907292,46.93,21.00,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2016,1,3,0.688837,14.488372,6616.406076,0.0,0.0,10.3,31.997083,46.93,21.55,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,2016,1,4,0.679545,22.477273,7367.750278,0.0,14.0,6.4,33.424583,46.93,20.90,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2016,1,5,0.768837,22.581395,7462.242014,0.0,39.0,4.4,33.053958,46.93,19.05,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [377]:
# identify response variable and predictors
X = df.drop(['TOTALDEMAND'], axis=1).values
y = df['TOTALDEMAND'].values

# Split the dataset into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [378]:
# Normalize the data

scaler = MinMaxScaler(feature_range=(0, 1))
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


print(X_train_scaled[:5])



[[0.66666667 0.27272727 0.         0.82446463 0.42215465 0.
  0.         0.18269231 0.37598142 0.46602747 0.60465116 0.
  0.         1.         0.         0.         0.         0.        ]
 [0.66666667 0.18181818 0.3        0.84998342 0.40512547 0.
  0.         0.53205128 0.30048508 0.64777414 0.57122093 0.
  1.         0.         0.         0.         0.         0.        ]
 [0.33333333 0.72727273 0.86666667 0.79107722 0.29346085 0.
  0.00125313 0.67628205 0.57016328 0.21803648 0.40843023 0.
  0.         0.         1.         0.         0.         0.        ]
 [0.66666667 0.90909091 0.1        0.70522388 0.31685972 0.
  0.         0.81089744 0.3494768  0.95154981 0.57994186 0.
  0.         1.         0.         0.         0.         0.        ]
 [0.5        0.90909091 0.9        0.46202322 0.43203417 0.
  0.         0.94871795 0.44530892 0.51957991 0.60755814 0.
  0.         0.         1.         0.         0.         0.        ]]


In [379]:
#Reshape the data for input to the LSTM model
#(This step is fit the data to 3D tensor format for LSTM model to process the sequential data efficiently 
#and capture any temporal dependencies in the data)


X_train = []
y_train = []
for i in range(1, len(X_train_scaled)):
    X_train.append(X_train_scaled[i-1:i, :])
    y_train.append(X_train_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2]))

X_test = []
y_test = []
for i in range(1, len(X_test_scaled)):
    X_test.append(X_test_scaled[i-1:i, :])
    y_test.append(X_test_scaled[i, 0])
X_test, y_test = np.array(X_test), np.array(y_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], X_train.shape[2]))

print(X_test[:5])





[[[0.5        0.18181818 0.1        0.69912935 0.43820075 0.
   0.         0.71794872 0.63006961 0.24762958 0.7252907  1.
   0.         0.         0.         0.         0.         0.        ]]

 [[0.         0.54545455 0.23333333 0.91631615 0.75629763 0.
   0.10526316 0.12820513 0.49293763 0.12883201 0.40406977 0.
   0.         0.         0.         1.         0.         0.        ]]

 [[0.83333333 0.27272727 0.03333333 0.80208333 0.29244527 1.
   0.         0.56730769 0.14612381 0.2686445  0.60174419 0.
   0.         0.         0.         1.         0.         0.        ]]

 [[0.66666667 1.         0.9        0.70696517 0.39942605 0.5
   0.00125313 0.5        0.20490106 0.64224669 0.7369186  1.
   0.         0.         0.         0.         0.         0.        ]]

 [[0.66666667 0.18181818 0.23333333 0.8492869  0.59099484 0.
   0.05889724 0.40384615 0.22160915 0.64777414 0.57122093 0.
   0.         0.         0.         0.         0.         1.        ]]]


In [411]:
# Build the LSTM model with three LSTM layers and one dense output layer

model = Sequential()
model.add(LSTM(units=64, activation='relu', return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=32, activation='tanh', return_sequences=True))
model.add(LSTM(units=16, activation='relu'))
model.add(Dense(units=1, activation='linear'))


optimizer = RMSprop(learning_rate=0.0001)
model.compile(loss='mean_squared_error', optimizer=optimizer)



In [412]:
# Train the LSTM model
model.fit(X_train, y_train, epochs=100, batch_size=32)




Epoch 1/100
52/52 [==============================] - 3s 3ms/step - loss: 0.2755
Epoch 2/100
52/52 [==============================] - 0s 3ms/step - loss: 0.2622
Epoch 3/100
52/52 [==============================] - 0s 3ms/step - loss: 0.2471
Epoch 4/100
52/52 [==============================] - 0s 2ms/step - loss: 0.2287
Epoch 5/100
52/52 [==============================] - 0s 2ms/step - loss: 0.2068
Epoch 6/100
52/52 [==============================] - 0s 2ms/step - loss: 0.1809
Epoch 7/100
52/52 [==============================] - 0s 2ms/step - loss: 0.1522
Epoch 8/100
52/52 [==============================] - 0s 2ms/step - loss: 0.1252
Epoch 9/100
52/52 [==============================] - 0s 2ms/step - loss: 0.1084
Epoch 10/100
52/52 [==============================] - 0s 2ms/step - loss: 0.1037
Epoch 11/100
52/52 [==============================] - 0s 2ms/step - loss: 0.1032
Epoch 12/100
52/52 [==============================] - 0s 2ms/step - loss: 0.1031
Epoch 13/100
52/52 [=================

In [413]:
# Evaluate the LSTM model

# Get model predictions on the test set
y_pred = model.predict(X_test)

# Calculate MSE and R-squared
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print('MSE: %.4f' % mse)
print('R-squared: %.4f' % r2)

MSE: 0.0983
R-squared: 0.0118
